In [1]:
!pip install beautifulsoup4 requests openpyxl


In [2]:
from bs4 import BeautifulSoup as bs
from google.colab import drive
import os
import re
import requests as rq
import json
import time
from openpyxl import Workbook
from typing import Optional, Union, Dict, List
import random
import math

In [5]:

import pandas as pd

from google.colab import drive

drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [6]:
cd /content/drive/MyDrive/Eudops_Project

/content/drive/MyDrive/Eudops_Project


This is the code for getting all the reviews from Saladlab

In [23]:
class Coupang:
    headers = {
        "authority": "weblog.coupang.com",
        "scheme": "https",
        "origin": "https://www.coupang.com",
        "sec-ch-ua-mobile": "?0",
        "sec-ch-ua-platform": "macOS",
        "user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Whale/3.20.182.14 Safari/537.36",
        "cookie": "PCID=31489593180081104183684; _fbp=fb.1.1644931520418.1544640325; gd1=Y; X-CP-PT-locale=ko_KR; MARKETID=31489593180081104183684; sid=03ae1c0ed61946c19e760cf1a3d9317d808aca8b; overrideAbTestGroup=%5B%5D; x-coupang-origin-region=KOREA; x-coupang-accept-language=ko_KR;"
    }

    @staticmethod
    def get_product_code(url: str) -> str:
        """Extract PRODUCT CODE from the provided URL."""
        return url.split('products/')[-1].split('?')[0]

    def get_user_info(self, user_uid: str, product_url: str) -> Dict[str, Union[str, int]]:
        url = 'https://www.coupang.com/vp/product/reviews/profile/' + user_uid
        self.headers['referer'] = product_url

        results = []
        with rq.Session() as session:
            response = session.get(url, headers=self.headers)

        soup = bs(response.text, 'html.parser')

        username_2 = soup.select_one('div.sdp-review__profile__article__info__name')
        if username_2 == None:
                username_2 = 'N/A'
        else :
            username_2 = username_2.text.strip()

        review_count = soup.select_one('div.js_reviewProfileModalHiddenTotalCount')
        if review_count == None:
              review_count = 0
        else:
          review_count = review_count.attrs['data-total-count']

        user_info = {
            'user_uid': user_uid,
            'username_2': username_2,
            'review_count': review_count
        }

        return user_info


    def fetch_reviews(self, user_uid: str, product_url: str, pages: int) -> List[List[Dict[str, Union[str, int]]]]:
        # product_code = self.get_product_code(product_url)

        #URL only containing the reviews, from the latest
        urls = [
            f'https://www.coupang.com/vp/product/reviews/profile/{user_uid}/reviews?page={page}&size=15'
            for page in range(0, pages + 1)
        ]

        #Needed for getting reviews from coupang
        self.headers['referer'] = product_url

        print(urls[0])

        results = []
        with rq.Session() as session:
            for url in urls:
                response = session.get(url, headers=self.headers)
                results.extend(self.parse_reviews(user_uid, response.text))
                time.sleep(random.randint(1,3))  # Be polite with the server
        return results

    def parse_reviews(self, user_uid: str, html: str) -> List[Dict[str, Union[str, int]]]:
        soup = bs(html, 'html.parser')
        articles = soup.select('div.sdp-review__profile__article__list__reviews')
        user_review_data = []

        for article in articles:
            # Review UID
            review_uid = article.select_one('div.sdp-review__profile__article__list__reviews__help')
            if review_uid == None:
              review_uid = 'N/A'
            else:
              review_uid = review_uid.attrs['data-review-id']

            # Rating
            rating = article.select_one('div.sdp-review__profile__article__list__reviews__star__orange')
            if rating == None:
                rating = 0
            else :
                rating = int(rating.attrs['data-rating'])

            # Product Name
            prod_name = article.select_one('div.sdp-review__profile__article__list__reviews__product__name')
            if prod_name == None or prod_name.text == '':
                prod_name = 'N/A'
            else:
                prod_name = prod_name.text.strip()

            # Date
            date = article.select_one('div.sdp-review__profile__article__list__reviews__star__date').text.strip()

            # Seller
            seller = article.select_one('div.sdp-review__profile__article__list__reviews__seller_name')
            if seller == None or seller.text == '':
                seller = 'N/A'
            else:
                seller = seller.text.strip()[5:]

            # Review Title/Headline
            headline = article.select_one('div.sdp-review__profile__article__list__reviews__title')
            if headline == None or headline.text == '':
                headline = 'N/A'
            else:
                headline = headline.text.strip()

            # Review Content
            review_content = article.select_one('div.sdp-review__profile__article__list__reviews__content')
            if review_content == None :
                review_content = 'N/A'
            else:
                review_content = re.sub('[\n\t]','',review_content.text.strip())

            # Survey Feedback
            answer = article.select_one('span.sdp-review__profile__article__list__reviews__survey__row__answer')
            if answer == None or answer.text == '':
                answer = 'N/A'
            else:
                answer = answer.text.strip()


            user_review_details = {
                'review_uid' : review_uid,
                'user_uid': user_uid,
                'rating': rating,
                'product_name': prod_name,
                'review_date': date,
                'seller_name': seller,
                'review_title': headline,
                'review_content': review_content,
                'survey_answer': answer
            }

            user_review_data.append(user_review_details)


        return user_review_data

In [16]:
def get_reviews_of_user(user_uid, product_url):

    coupang_scraper = Coupang()

    user_info = coupang_scraper.get_user_info(user_uid, product_url)

    #get number of pages needed for user review collection
    pages = math.ceil(int(user_info['review_count']) / 15)
    print(pages)

    reviews = coupang_scraper.fetch_reviews(user_uid, product_url, pages)
    df_reviews = pd.DataFrame(reviews)

    return df_reviews


In [18]:
def collect_reviews_of_users_from_product(product_url, user_uid_list):
    for user_uid in user_uid_list:
        df_reviews = get_reviews_of_user(str(user_uid), product_url)
        df_reviews.to_csv(f'/content/drive/My Drive/Eudops_Project/Coupang_Collected_User_Reviews/{user_uid}.csv', mode='a', index=False, header=True)

        print(f"User ID: {user_uid} saved in Google Drive")


In [ ]:
product_reviews = pd.read_csv('/content/drive/My Drive/Eudops_Project/Coupang_Collected_Reviews/LaRive.csv')
user_uid_list = product_reviews['user_uid'].unique().tolist()

product_url = "https://www.coupang.com/vp/products/546702?itemId=3305967319&searchId=feed-6028e78f15fa4261967295c4a1a367a4-view_together_ads-P7230281913&vendorItemId=73999864881&sourceType=SDP_ADS&clickEventId=0d2ef1b0-10da-11ef-ab84-9b61cfa07e00&isAddedCart="

collect_reviews_of_users_from_product(product_url, user_uid_list)

33
https://www.coupang.com/vp/product/reviews/profile/108677123/reviews?page=0&size=15
User ID: 108677123 saved in Google Drive
55
https://www.coupang.com/vp/product/reviews/profile/17615181/reviews?page=0&size=15
User ID: 17615181 saved in Google Drive
11
https://www.coupang.com/vp/product/reviews/profile/35478249/reviews?page=0&size=15
User ID: 35478249 saved in Google Drive
0
https://www.coupang.com/vp/product/reviews/profile/4087235/reviews?page=0&size=15
User ID: 4087235 saved in Google Drive
12
https://www.coupang.com/vp/product/reviews/profile/63842276/reviews?page=0&size=15
User ID: 63842276 saved in Google Drive
29
https://www.coupang.com/vp/product/reviews/profile/12695361/reviews?page=0&size=15
User ID: 12695361 saved in Google Drive
33
https://www.coupang.com/vp/product/reviews/profile/106691199/reviews?page=0&size=15
User ID: 106691199 saved in Google Drive
4
https://www.coupang.com/vp/product/reviews/profile/108813187/reviews?page=0&size=15
User ID: 108813187 saved in Goo

In [20]:
print(user_uid_list)

[108677123, 17615181, 35478249, 4087235, 63842276, 12695361, 106691199, 108813187, 108787809, 124300, 23489259, 112209650, -31714870, 115006632, -26890392, 104874797, 107549867, 109505065, 903019, 17577769, 106443025, 156280260, 114780473, 19022859, 16151692, 423957, 104128832, 146220798, 101487811, 103759259, 42329711, 8648353, 104589051, 112618949, 9917682, 43356903, 109883726, 2059136, 9763282, 101411960, -81967175, 28137635, 114665897, 21433060, 29412758, 47052896, 110924841, 140398892, 106252981, 21187427, 3196753, 29951097, 103341673, 103780400, 22881478, 464848, 118912370, 45319087, 11909103, 107555561, 12980981, 141156573, 38807145, -38474935, 26998237, 113515441, 36705542, 105927951, -84225190, 36148758, 112559720, 49446590, 107317916, 111930237, 613086, 5785795, 11743720, 107414317, 57701349, 36614854, 111685431, 13696176, 108162827, 47600926, -130207815, 152541195, 100334751, 107429792, 112309222, -47241685, 7179376, 114262054, 5309988, 114609729, 109508572, 115025860, 10950

151
https://www.coupang.com/vp/product/reviews/profile/4087235/reviews?page=0&size=15


KeyboardInterrupt: 

In [1]:
ls

sample_data/


In [2]:
cd ../

/


In [3]:
ls

bin@                        datalab/  kaggle/  libx32@                   opt/   sbin@  tools/
boot/                       dev/      lib@     media/                    proc/  srv/   usr/
content/                    etc/      lib32@   mnt/                      root/  sys/   var/
cuda-keyring_1.0-1_all.deb  home/     lib64@   NGC-DL-CONTAINER-LICENSE  run/   tmp/
